In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K



import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
######### VGG19 parameters ########
dontFreezeLast = 2;

patience = 30;

loadWeights = False;
saveWeights = False;


tensorboard_dir = '../tb/catsdogs/try_120x2_90x2_120';
if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)    
checkPointPath = tensorboard_dir + '/best_weights.hdf5';

####################################

gpuName = '/device:GPU:1'
tensorboardFlag = True;
workers = 10;
histogram_freq = 0;

batchSize = 64;
epochs = 100;
validation_size=0.3;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

('Training images: ', 10222)
('Test images: ', 10357)


In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 90

In [8]:
x_train = []
y_train = []
x_test = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:30<00:00, 334.94it/s]


In [10]:
 for f in tqdm(df_test['id'].values):
     img = cv2.imread('../input/test/{}.jpg'.format(f))
     x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:30<00:00, 343.54it/s]


In [11]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

('Number of classes: ', 120)


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
# Create the base pre-trained model
# Can't download weights in the kernel
with tf.device(gpuName):
    
 
    if K.image_data_format() == 'channels_first':
        input_shape = (3, im_size, im_size)
    else:
        input_shape = (im_size, im_size, 3)

     
    
    model = Sequential()
    model.add(Conv2D(120, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(120, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(90, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(90))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(120))
    model.add(Activation('softmax'))

   
    ##### Mattia's model #####
    # Add a new top layer
    #x = base_model.output
    #x = Flatten()(x)
    #x = Dense(1024,activation='relu')(x)
    #x = Dense(512,activation='relu')(x)
    #predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    #model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    #for i in range(len(base_model.layers)-dontFreezeLast):
        #base_model.layers[i].trainable = False

    #if loadWeights:
    #    model.load_weights(checkPointPath)
    ##### Mattia's model #####
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=patience,
        verbose=1));
    if saveWeights:
        callbacks_list.append(keras.callbacks.ModelCheckpoint(
            checkPointPath,
            monitor='val_acc',
            verbose=1,
            save_best_only=True,
            mode='max',
            save_weights_only=True))
    if tensorboardFlag:
        callbacks_list.append(keras.callbacks.TensorBoard(
                log_dir=tensorboard_dir,
                histogram_freq=histogram_freq,
                write_graph=False,
                write_images=False));
        print('Tensorboard activated in directory: ',tensorboard_dir)
    else:
        print('Tensorboard NOT activated')


    model.summary()

('Tensorboard activated in directory: ', '../tb/catsdogs/try_120x2_90x2_120')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 88, 88, 120)       3360      
_________________________________________________________________
activation_1 (Activation)    (None, 88, 88, 120)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 44, 44, 120)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 42, 42, 120)       129720    
_________________________________________________________________
activation_2 (Activation)    (None, 42, 42, 120)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 21, 120)       0         
________________________________________________________________

In [16]:
def generator(X,Y,batch_size):
    batch_features = np.ndarray(shape=(batch_size,) + X.shape[1:],
                                dtype=X.dtype);
    batch_labels = np.ndarray(shape=(batch_size,) + Y.shape[1:],
                                dtype=Y.dtype)
    N = X.shape[0];
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= np.random.choice(N,1)
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels

In [17]:
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size = batchSize,
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)

# steps_per_epoch = round(X_train.shape[0]/batchSize)
# model.fit_generator(generator(X_train,Y_train,batchSize),
#                    steps_per_epoch=steps_per_epoch,
#                    epochs=epochs,
#                    verbose=1,
#                    callbacks=callbacks_list,
#                    validation_data=(X_valid,Y_valid),
#                    workers=workers,
#                    use_multiprocessing=True)

Train on 7155 samples, validate on 3067 samples
Epoch 1/100
7155/7155 [==============================] - 18s 2ms/step - loss: 4.7891 - acc: 0.0094 - val_loss: 4.7862 - val_acc: 0.0104
Epoch 2/100
7155/7155 [==============================] - 16s 2ms/step - loss: 4.7848 - acc: 0.0103 - val_loss: 4.7834 - val_acc: 0.0088
Epoch 3/100
7155/7155 [==============================] - 16s 2ms/step - loss: 4.7756 - acc: 0.0130 - val_loss: 4.7605 - val_acc: 0.0124
Epoch 4/100
7155/7155 [==============================] - 16s 2ms/step - loss: 4.7397 - acc: 0.0164 - val_loss: 4.7166 - val_acc: 0.0212
Epoch 5/100
7155/7155 [==============================] - 16s 2ms/step - loss: 4.6553 - acc: 0.0250 - val_loss: 4.6436 - val_acc: 0.0258
Epoch 6/100
7155/7155 [==============================] - 16s 2ms/step - loss: 4.5704 - acc: 0.0337 - val_loss: 4.5818 - val_acc: 0.0300
Epoch 7/100
7155/7155 [==============================] - 16s 2ms/step - loss: 4.4611 - acc: 0.0450 - val_loss: 4.5606 - val_acc: 0.0329


Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [18]:
 preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - 8s 765us/step


In [19]:
 #sub = pd.DataFrame(preds)
 ## Set column names to those generated by the one-hot encoding earlier
 #col_names = one_hot.columns.values
 #sub.columns = col_names
 ## Insert the column id from the sample_submission at the start of the data frame
 #sub.insert(0, 'id', df_test['id'])
 #sub.head(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,1.297472e-06,4.442717e-03,7.605385e-03,1.566002e-04,2.225735e-05,4.872090e-05,2.205260e-05,6.250631e-03,3.766111e-02,...,9.232412e-05,1.500094e-03,5.045601e-06,2.603026e-02,2.233342e-04,9.833261e-05,1.249142e-03,1.963588e-04,1.295604e-02,1.063086e-05
1,00102ee9d8eb90812350685311fe5890,3.314704e-14,4.430033e-11,5.807886e-10,9.932768e-11,9.038590e-08,2.791932e-14,5.310895e-13,9.601794e-10,1.269792e-08,...,4.282991e-04,5.244528e-13,9.881392e-07,1.111853e-05,1.237807e-06,1.633271e-05,3.322119e-02,2.085098e-03,8.081533e-05,1.691316e-11
2,0012a730dfa437f5f3613fb75efcd4ce,1.215672e-06,2.779796e-03,7.978015e-04,9.666760e-07,3.370161e-03,5.763080e-02,7.078919e-05,1.952740e-02,1.332434e-03,...,1.640732e-05,7.101269e-04,1.517099e-05,1.988362e-04,1.046027e-03,2.253631e-02,7.047165e-02,4.114278e-05,7.406237e-04,6.106808e-04
3,001510bc8570bbeee98c8d80c8a95ec1,7.474050e-06,4.874651e-05,7.420365e-08,4.055916e-08,1.703381e-04,1.609215e-10,5.232832e-12,1.614717e-07,6.839575e-11,...,3.547481e-06,9.951367e-12,5.820880e-19,9.849149e-09,4.350561e-03,5.702882e-09,5.528812e-06,6.331813e-15,2.984793e-08,6.961228e-06
4,001a5f3114548acdefa3d4da05474c2e,5.400772e-13,7.776105e-05,1.611469e-03,2.344649e-10,1.754181e-04,1.477825e-04,7.122642e-10,7.279136e-03,3.529318e-04,...,1.231418e-04,2.373464e-02,4.256888e-07,3.072863e-05,2.878058e-06,4.896322e-04,1.496229e-05,2.171360e-02,6.986241e-05,6.767849e-09
5,00225dcd3e4d2410dd53239f95c0352f,2.326147e-03,5.083885e-03,8.298901e-05,4.862810e-03,9.304270e-05,1.993373e-04,3.404737e-04,6.931429e-04,1.736546e-06,...,4.757400e-04,1.132760e-05,1.737656e-06,1.033148e-05,1.454001e-02,7.841020e-05,2.183276e-05,4.181068e-05,4.302509e-06,2.575408e-04
6,002c2a3117c2193b4d26400ce431eebd,3.793781e-13,1.272760e-01,4.244606e-05,1.992415e-08,1.625738e-04,2.127391e-06,9.627821e-05,7.745483e-06,5.475042e-05,...,2.250461e-04,1.057108e-17,2.743617e-14,1.224260e-07,4.089502e-09,1.223843e-06,1.557282e-13,3.606922e-09,6.115328e-07,5.125562e-12
7,002c58d413a521ae8d1a5daeb35fc803,1.638891e-10,2.039429e-26,3.486952e-18,1.248922e-04,2.424193e-11,3.526092e-10,1.208586e-09,8.541535e-17,8.050536e-13,...,7.269767e-07,4.061977e-06,1.521300e-08,1.334860e-11,4.995933e-11,7.774266e-07,1.011722e-04,8.441501e-11,1.825765e-16,3.698782e-10
8,002f80396f1e3db687c5932d7978b196,2.474975e-05,4.803732e-02,1.600201e-03,3.481425e-05,2.376630e-03,4.529352e-06,4.871333e-05,6.400200e-04,1.770719e-04,...,2.481867e-04,1.184384e-04,2.908283e-06,2.182896e-01,3.587548e-05,1.965045e-06,9.695903e-08,4.940421e-04,5.944329e-05,3.883719e-06
9,0036c6bcec6031be9e62a257b1c3c442,8.457770e-06,7.012788e-04,9.127921e-07,3.011074e-02,3.429301e-04,1.352615e-08,1.774099e-03,1.050228e-02,3.843151e-03,...,3.833169e-01,4.258048e-10,8.521110e-05,2.487504e-07,1.217846e-03,7.831063e-06,2.238648e-05,3.077477e-05,1.438444e-04,2.034963e-04
